In [76]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns

sns.set_style('ticks')

from tqdm import tqdm
from itertools import (count, product)
from scipy.stats import poisson
from black_jack_env import BlackJackEnv, calc_hand_value_usable_ace
from black_jack_agent import BlackJackAgent, get_state_idx

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
## Training Loop

epsilon = 0.05
#initialization
policy = np.ones((2, 200)) * 0.5
q = np.zeros((2, 200))

# loop
for ep_idx in range(1000):
    agent = BlackJackAgent(policy)
    env = BlackJackEnv(seed=None)
    result, player_hand, dealer_card = env.deal_cards()
    state_idxs = []
    actions = []
    while result is None:
        player_value, num_aces = calc_hand_value_usable_ace(player_hand)
        has_usable_ace = num_aces > 0
        state_idx = get_state_idx(has_usable_ace, player_value, dealer_card)
        action = agent.take_action(state_idx)
        state_idxs.append(state_idx)
        actions.append(action)
        if action == 0:
            # this is hit
            result, player_hand = env.hit()
        else:
            result = env.stick()
    for state_idx, action in zip(state_idxs, actions):
        q_new = (q[action, state_idx] * ep_idx + result) / (ep_idx + 1)
        q[action, state_idx] = q_new
        a_star = np.argmax(q[:, state_idx])
        policy[:, state_idx] = epsilon / 2
        policy[a_star, state_idx] = 1 - epsilon + epsilon / 2
    
    